In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from ucimlrepo import fetch_ucirepo 

In [ ]:
# Create a model class that inherits nn.Module
class Model(nn.Module):
    # input layer (4 different types of flower features)
    # hidden layer 1 (some number of neurons)
    # hidden layer 2 (some number of neurons)
    # output layer (3 different types of possible flowers)
    def __init__(self, in_features=4, h1=7, h2=9, out_features=3):
        super().__init__() # instantiate our nn.Module
        self.fc1 = nn.Linear(in_features, h1)
        #self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h1, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = F.relu(self.out(x))

        return x


In [ ]:
# Pick a manual seed for randomization
torch.manual_seed(41)
model = Model()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
my_df = fetch_ucirepo(id=53)

# data as pandas dataframes
X = my_df.data.features
Y = my_df.data.targets


# Convert these values to numpy arrays
X = X.values
Y = Y.values

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Encode target labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 41)

# convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# convert Y labels to tensors
Y_train = torch.LongTensor(Y_train)
Y_test = torch.LongTensor(Y_test)

In [ ]:
# Set the criterion of model to measure the error

criterion = nn.CrossEntropyLoss()
# Choose Adam optimizer, lr = learning rate (if error does not go down after a bunch of iterations (epochs) lower learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train our model
# Epochs? (one run through all the training data in our network)
epochs = 130
losses = []
for i in range(epochs):
    # go forward and get a prediction
    y_pred = model(X_train) # Get predicited results

    # measure the loss/error, will be high at first
    loss = criterion(y_pred, Y_train) # predicted value vs the y_train

    # keep track of our losses
    losses.append(loss.detach().numpy())

    # print every 10 epochs
    if(i % 10 == 0):
        print(f"Epoch: {i} and loss: {loss}")
    
    # Do backpropogation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# Graph it 
plt.plot(range(epochs), losses)
plt.ylabel("loss/error")
plt.xlabel("epochs")

In [ ]:
# Evaluate Model on Test Data Set (validate model on test set)
with torch.no_grad():
    Y_eval = model.forward(X_test) # X_test are features from our test set, y_eval will be predictions
    loss = criterion(Y_eval, Y_test) # Find the loss or error
loss

In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = model.forward(data)

        print(f"{i+1} {str(y_val)} \t {Y_test[i]} \t {y_val.argmax().item()}")

        if y_val.argmax().item() == Y_test[i]:
            correct += 1

print(f"we got {correct} correct")

In [ ]:
# how to test with new data
new_iris = torch.tensor([4.7, 3.2, 1.3, 0.2])
with torch.no_grad():
    print(model(new_iris))

In [ ]:
# save the model
torch.save(model.state_dict(), 'my_awesome_model.pt')

In [ ]:
# load the saved model
new_model = Model()
new_model.load_state_dict(torch.load('my_awesome_model.pt'))

In [ ]:
# make sure it loaded correctly
new_model.eval()